In [1]:
import os
from glob import glob
from tqdm import tqdm
import numpy as np
import pandas as pd

In [3]:
BASE_DIR = '/root/class/cmu/LSMA/11775-hws/spring2022/hw1'
DATA_DIR = '/shared/youngkim/dataset'
LABEL_DIR = os.path.join(DATA_DIR, 'labels')

In [11]:
'selected.mfcc.csv' in os.listdir(os.path.join(DATA_DIR, 'mfcc'))

False

In [47]:
print(os.listdir(DATA_DIR))
'''
1. videos : video data (.mp4) * 8249
2. mp3 | wav : audio data from video (.mp3 / .wav) * 8245 ? (where is 4 file compared to videos?)
    - wav : channel 1, sample rate 44100
    - mp3 : channel 1, sample rate 22050
3. mfcc : MFCCs from wav (.csv) * 8245 & 'selected.mfcc.csv' * 1
    -> train K-Means(k=50) using 'selected.mfcc.csv' => return models/kmeans.50.model (CODEBOOK) (k can be changed)
4-1. bof : 50 features from MFCC & CODEBOOK (.csv) * 8245 (# of features == # of cluster)
4-2. raw : out features from SoundNet(model) & mp3(input) (.npy) * 8245
    - .npy.item().keys() -> ['conv7', 'y_obj', 'y_scns']
    - conv7.shape = 1, 1024, sequence length, 1
    - y_obj.shape = 1, 1000, sequence length, 1
    - y_scns.shape = 1, 401, sequence length, 1
                    (batch size, features, sequence length, output dimension?)
5. avg_pooling/* : features after average pooling (.csv) * 8245
    - conv7 : 1024
    - y_obj : 1000
    - y_scns : 401
'''

['avg_pooling', 'videos', 'raw', 'bof', 'mp3', 'wav', 'labels', 'mfcc', 'models']


"\n1. videos : video data (.mp4) * 8249\n2. mp3 | wav : audio data from video (.mp3 / .wav) * 8245 ? (where is 4 file compared to videos?)\n    - wav : channel 1, sample rate 44100\n    - mp3 : channel 1, sample rate 22050\n3. mfcc : MFCCs from wav (.csv) * 8245 & 'selected.mfcc.csv' * 1\n    -> train K-Means(k=50) using 'selected.mfcc.csv' => return models/kmeans.50.model (CODEBOOK) (k can be changed)\n4-1. bof : 50 features from MFCC & CODEBOOK (.csv) * 8245 (# of features == # of cluster)\n4-2. raw : out features from SoundNet(model) & mp3(input) (.npy) * 8245\n    - .npy.item().keys() -> ['conv7', 'y_obj', 'y_scns']\n    - conv7.shape = 1, 1024, sequence length, 1\n    - y_obj.shape = 1, 1000, sequence length, 1\n    - y_scns.shape = 1, 401, sequence length, 1\n                    (batch size, features, sequence length, output dimension?)\n5. avg_pooling/* : features after average pooling (.csv) * 8245\n    - conv7 : 1024\n    - y_obj : 1000\n    - y_scns : 401\n"

In [29]:
for i in range(10):
    sample_npy = os.listdir(os.path.join(DATA_DIR, 'raw'))[i]
    sample_npy = np.load(os.path.join(DATA_DIR, f'raw/{sample_npy}'), allow_pickle=True)
    sample_npy = sample_npy.item()
    print(sample_npy['y_scns'].shape)

(1, 401, 7, 1)
(1, 401, 4, 1)
(1, 401, 4, 1)
(1, 401, 4, 1)
(1, 401, 4, 1)
(1, 401, 6, 1)
(1, 401, 4, 1)
(1, 401, 4, 1)
(1, 401, 7, 1)
(1, 401, 7, 1)


In [35]:
os.listdir(os.path.join(DATA_DIR, 'avg_pooling/y_scns'))[0]

'LTYzMTY1ODM5NTYxMjk3NTMyODc=.csv'

In [4]:
len(os.listdir(os.path.join(DATA_DIR, 'bof')))

8245

In [23]:
raw_data_paths = glob(os.path.join(DATA_DIR, 'raw/*'))
raw_sample = np.load(raw_data_paths[0], allow_pickle=True).item()
raw_sample['conv7'].shape, raw_sample['y_obj'].shape, raw_sample['y_scns'].shape

((1, 1024, 21, 1), (1, 1000, 7, 1), (1, 401, 7, 1))

In [46]:
avg_pool_conv7_paths = glob(os.path.join(DATA_DIR, 'avg_pooling/conv7/*'))
avg_pool_conv7_sample = pd.read_csv(avg_pool_conv7_paths[0], header=None)
avg_pool_conv7_sample.shape

(1024, 1)

In [43]:
avg_pool_y_obj_paths = glob(os.path.join(DATA_DIR, 'avg_pooling/y_obj/*'))
avg_pool_y_obj_sample = pd.read_csv(avg_pool_y_obj_paths[0], header=None)
avg_pool_y_obj_sample.shape

(1000, 1)

In [45]:
avg_pool_y_scns_paths = glob(os.path.join(DATA_DIR, 'avg_pooling/y_scns/*'))
avg_pool_y_scns_sample = pd.read_csv(avg_pool_y_scns_paths[0], header=None)
avg_pool_y_scns_sample.shape

(401, 1)

In [4]:
os.listdir(LABEL_DIR)

['train_val.csv', 'cls_map.csv', 'test_for_students.csv']

In [62]:
train_val_df = pd.read_csv(os.path.join(LABEL_DIR, 'train_val.csv'))
train_val_df

,Id,Category
0,NjUzNjkxMTk0MzI5NDYwNTA2NA==,0
1,LTcxNTE1MDY1Mjc5NzMxMTExNDI=,0
2,NTMyNTQxODcwODc5MjU2NDk4NA==,0
3,LTY0NjAxNjcyNTUyNDI5Mzc2MTE=,0
4,NTgzMjc4Mzk0MjQwMzYyMjA1NQ==,0
...,...,...
7495,LTUxMTQ5Mjg2OTQyNjgyMTc1NzA=,14
7496,NDA2NjM0NDg5MDc3NzM3NjQ5OA==,14
7497,LTEwMzgyODQzNTYwMjM5NDY3NzI=,14
7498,LTQzOTI1MDQ1OTQ3NzUyMjQ4Mg==,14


In [10]:
test_df = pd.read_csv(os.path.join(LABEL_DIR, 'test_for_students.csv'))
test_df

,Id,Category
0,LTExODM2Mzc0ODQyOTc1ODE4NDM=,NaN
1,LTUwNDU3NzgyNjE2Mzk0OTU1NjQ=,NaN
2,ODU3OTE0MDU5NzM5NDI2MDQ2,NaN
3,LTQ4OTE2NDk5ODk4MTA3NDY5Mzg=,NaN
4,MjkzNzMwNzkxNTg5NDA1Mjk3NQ==,NaN
...,...,...
744,Nzk0NzA0MjYxMTg3NDkxNjQ4OQ==,NaN
745,LTMzMjA0NjkyNzYwMDQ0NDUyNzY=,NaN
746,LTQxNDgxNjU0MzY3OTQzNzIzMTc=,NaN
747,NjQyMjk5MDM5OTgwNzY2NDU1MQ==,NaN


In [13]:
video_names = os.listdir(os.path.join(DATA_DIR, 'videos'))
video_names = [x.split('.m')[0] for x in video_names]
video_names[:4], len(video_names)

(['NjM0NzQ4NzY1NDk5NTU2MDIwMg==',
  'LTU4NzEwMDUxODUyNjk2MzUyMDE=',
  'MzY0MDY5NDQxMjE4NDkwNDYwMw==',
  'LTg4ODI5NDk0MjQzMjkzOTUxNjc='],
 8249)

In [15]:
wav_names = os.listdir(os.path.join(DATA_DIR, 'wav'))
wav_names = [x.split('.w')[0] for x in wav_names]
wav_names[:4], len(wav_names)

(['MzAxOTUwOTU4MTExNjMxNTE4OQ==',
  'MjU5MzIwNDg2MDQ1MDM1Njg3OQ==',
  'MjM3NDc0NjAxNTQzOTE3MzE0OA==',
  'NjE5ODk5NjM1NTUxNjE4NTk2OA=='],
 8245)

In [18]:
v_set = set(video_names)
w_set = set(wav_names)
no_audios = v_set - w_set
no_audios

{'LTQ5ODI3NjU5MTQ3OTQ4NTAwOQ==',
 'LTgxOTM5Mzg2MTMwNzM4NjQzNzg=',
 'Mjg0MDgxODgzMDUyOTk3NjE2OA==',
 'NTkxNzA4MjE4OTM1ODg4NTYxOA=='}

In [63]:
for no_audio in no_audios:
    train_val_df = train_val_df.drop(index=train_val_df[train_val_df.Id == no_audio].index)

In [64]:
len(train_val_df)

7496

In [65]:
train_val_df.Category.value_counts()

0     500
1     500
3     500
4     500
5     500
7     500
8     500
9     500
10    500
11    500
12    500
13    500
6     499
14    499
2     498
Name: Category, dtype: int64

In [50]:
from sklearn.model_selection import StratifiedKFold


In [66]:
train_val_df = train_val_df.reset_index(drop=True)
gkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, ( _, val_) in enumerate(gkf.split(X=train_val_df, y=train_val_df.Category)):
      train_val_df.loc[val_ , "kfold"] = fold

train_val_df.head()

,Id,Category,kfold
0,NjUzNjkxMTk0MzI5NDYwNTA2NA==,0,3.0
1,LTcxNTE1MDY1Mjc5NzMxMTExNDI=,0,0.0
2,NTMyNTQxODcwODc5MjU2NDk4NA==,0,3.0
3,LTY0NjAxNjcyNTUyNDI5Mzc2MTE=,0,1.0
4,NTgzMjc4Mzk0MjQwMzYyMjA1NQ==,0,1.0


In [67]:
train_val_df[train_val_df.kfold==4.0].Category.value_counts()

0     100
1     100
3     100
4     100
5     100
6     100
7     100
8     100
9     100
10    100
11    100
12    100
13    100
14    100
2      99
Name: Category, dtype: int64

In [68]:
train_val_df.to_csv(os.path.join(LABEL_DIR, 'kfold_df.csv'), index=False)